In [ ]:
# Suppress Deprecation and Incorrect Usage Warnings
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import datetime as dt
import time
import gc
from dateutil.relativedelta import *

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

%matplotlib inline

from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgbm
import gc

from sklearn.linear_model import Lasso, ElasticNet, RANSACRegressor, Ridge
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier,ExtraTreesClassifier
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,GradientBoostingRegressor,ExtraTreesRegressor
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.svm import LinearSVR,SVR
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, log_loss
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from itertools import product

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from mpl_toolkits.axes_grid1 import host_subplot
import mpl_toolkits.axisartist as AA
%matplotlib inline

import seaborn as sns

### seaborn에서 한글 나오게하기
sns.set(font="New Gulim")

import zipfile
import stacking
from sklearn import preprocessing


# Load Data

In [ ]:
INPUT_FOLDER = './data'
train_origin = pd.read_csv('{}/train.csv'.format(INPUT_FOLDER))
test_origin = pd.read_csv('{}/test.csv'.format(INPUT_FOLDER))
print(train_origin.shape, test_origin.shape)

test_origin['Survived'] = 0
test_id = test_origin['PassengerId'][0]
print(test_id)

test_origin['Survived'] = 0
test_id = test_origin['PassengerId'][0]
print(test_id)

data = pd.concat([train_origin, test_origin])
print(data.shape)

data['Title'] = data['Name'].str.split(", ", expand=True)[1].str.split(".", expand=True)[0]
major_titles = data['Title'].value_counts()[:4].index.tolist()

map_title_misc = {'Don': 'Mrs', 'Dona': 'Mrs',  'Capt': 'Mr', 'Col': 'Mr', 'Jonkheer': 'Mr',
                  'Lady': 'Mrs', 'Major': 'Mr', 'Mlle': 'Miss', 'Mme': 'Mrs', 'Rev': 'Mrs', 'Rev': 'Mrs',
                  'Sir' : 'Mr', 'the Countess': 'Mrs', 'Ms': 'Miss'}

def clean_title(row):
    title = row['Title']
    if title in map_title_misc.keys():
        row['Title'] = map_title_misc[title]
    elif title == 'Dr':
        sex = row['Sex']
        row['Title'] = 'Mr' if sex == 'mail' else 'Mrs'
    return row

data = data.apply(clean_title, axis=1)

def cabin_class(row):
    cabin = str(row['Cabin'])
    row['CabinClass'] = np.nan
    if cabin is not None:
        row['CabinClass'] = cabin[0]
    return row

data = data.apply(cabin_class, axis=1)

data.drop(columns=['Cabin', 'Name', 'Ticket'], inplace=True)

data['Embarked'].fillna(data['Embarked'].mode()[0], inplace = True)
data['Fare'].fillna(data['Fare'].median(), inplace = True)
data['Age'].fillna(data['Age'].median(), inplace = True)

categorical_features = ['Sex', 'Title', 'Embarked']
def category_to_code(df, categorical_features):
    df_copy = df.copy()
    for feature in categorical_features:
        df_copy[feature] = df_copy[feature].astype('category').cat.codes
    return df_copy

data_code = category_to_code(data, categorical_features)
data_code.head()

In [ ]:
data_dummy = pd.get_dummies(data)
data_dummy.head()

In [ ]:
df = data_dummy.loc[data_dummy['PassengerId'] < test_id]
df.drop(columns='PassengerId', inplace=True)

test = data_dummy.loc[data_dummy['PassengerId'] >= test_id]
test.drop(columns=['PassengerId', 'Survived'], inplace=True)

In [ ]:
y_train = df['Survived']
x_train = df.drop(columns=['Survived'])
x_test = test
print(x_train.shape, y_train.shape, x_test.shape)

In [ ]:
from imp import reload
reload(stacking)

# Stacking Model

In [ ]:
et_params = {
    'criterion':'gini', 'max_leaf_nodes':30, 'n_estimators':1000, 'min_impurity_split':0.0000001,
    'max_features':0.6, 'max_depth':10, 'min_samples_leaf':20, 'min_samples_split':2,
    'min_weight_fraction_leaf':0.0, 'bootstrap':True,
    'random_state':1, 'verbose':False }

et_model = stacking.SklearnWrapper(clf = ExtraTreesClassifier, params=et_params)

et_train, et_test = stacking.get_oof(et_model, x_train, y_train, x_test, accuracy_score, n_folds=5)

### Param

In [ ]:
lgbm_params1 = {
    'boosting':'gbdt', 'num_leaves':10, 'learning_rate':0.01, 'min_sum_hessian_in_leaf':0.1,
    'max_depth':4, 'feature_fraction':0.5, 'min_data_in_leaf':4, 'poission_max_delta_step':0.7,
    'bagging_fraction':0.8, 'min_gain_to_split':0, 'scale_pos_weight':1.0,
    'lambda_l2':0.1, 'lambda_l1':0.1, 'huber_delta':1.0, 'bagging_freq':1,
    'objective':'binary', 'seed':1, 'categorical_feature':0, 'xgboost_dart_mode':False,
    'drop_rate':0.1, 'skip_drop':0.5, 'max_drop':50, 'top_rate':0.1, 'other_rate':0.1,
    'max_bin':255, 'min_data_in_bin':50, 'bin_construct_sample_cnt':1000000,
    'two_round':False, 'uniform_drop':False,'metric': 'binary_logloss','threads':6
}

lgbm_params2 = {
    'boosting':'gbdt', 'num_leaves':24,'learning_rate':0.03, 'min_sum_hessian_in_leaf':0.1,
    'max_depth':6, 'feature_fraction':0.5, 'min_data_in_leaf':50, 'poission_max_delta_step':0.7, 'bagging_fraction':0.8,
    'min_gain_to_split':0, 'scale_pos_weight':1.0, 'lambda_l2':0.1, 'lambda_l1':0.1, 'huber_delta':0.05,
    'bagging_freq':1, 'objective':'binary', 'seed':1, 'categorical_feature':0 ,'xgboost_dart_mode':False, 'drop_rate':0.1,
    'skip_drop':0.5, 'max_drop':50, 'top_rate':0.1, 'other_rate':0.1, 'max_bin':255, 'min_data_in_bin':50,
    'bin_construct_sample_cnt':1000000, 'two_round':False, 'uniform_drop':False,'metric': 'binary_logloss','threads':6
}

lgbm_params3 = {
    'boosting':'gbdt', 'num_leaves':28, 'learning_rate':0.03, 'min_sum_hessian_in_leaf':0.1, 'max_depth':7,
    'feature_fraction':0.6, 'min_data_in_leaf':70, 'poission_max_delta_step':0.7, 'bagging_fraction':0.8,
    'min_gain_to_split':0, 'scale_pos_weight':1.0, 'lambda_l2':0.1, 'lambda_l1':0.1, 'fair_c':0.01, 'bagging_freq':1,
    'objective':'binary', 'seed':1, 'categorical_feature':0, 'xgboost_dart_mode':False, 'drop_rate':0.1, 'skip_drop':0.5,
    'max_drop':50, 'top_rate':0.1, 'other_rate':0.1, 'max_bin':255, 'min_data_in_bin':50, 'bin_construct_sample_cnt':1000000,
    'two_round':False, 'uniform_drop':False,'metric': 'binary_logloss','threads':6
}

lgbm_params4 = {
    'boosting':'gbdt', 'num_leaves':16, 'learning_rate':0.003, 'min_sum_hessian_in_leaf':0.1, 'max_depth':7,
    'feature_fraction':0.5, 'min_data_in_leaf':70, 'poission_max_delta_step':0.7, 'bagging_fraction':0.8, 
    'min_gain_to_split':0, 'scale_pos_weight':1.0, 'lambda_l2':0.1, 'lambda_l1':0.1, 'bagging_freq':1, 'objective':'binary',
    'seed':1, 'categorical_feature':0, 'xgboost_dart_mode':False, 'drop_rate':0.1, 'skip_drop':0.5, 'max_drop':50, 
    'top_rate':0.1,'other_rate':0.1, 'max_bin':255, 'min_data_in_bin':50, 'bin_construct_sample_cnt':1000000, 
    'two_round':False,'uniform_drop':False,'metric': 'binary_logloss','threads':6
}

xgb_params1 = {
    'booster':'gbtree', 'objective':'binary:logistic', 'max_leaves':0, 'eta':0.02, 'gamma':1,
    'max_depth':4, 'colsample_bylevel':1.0, 'min_child_weight':4.0, 'max_delta_step':0.0, 'subsample':0.8, 
    'colsample_bytree':0.5,'scale_pos_weight':1.0, 'alpha':1.0, 'lambda':5.0, 'seed':1
}

xgb_params2 = {
    'booster':'gblinear', 'objective':'binary:logistic', 'max_leaves':0, 'eta':0.1,'gamma':1,
    'max_depth':4, 'colsample_bylevel':1.0, 'min_child_weight':4.0, 'max_delta_step':0.0, 'subsample':0.8, 
    'colsample_bytree':0.5,'scale_pos_weight':1.0, 'alpha':10.0, 'lambda':1.0, 'seed':1
}

sgd_param = {
    'loss':'huber','penalty':'l2','alpha':1,'l1_ratio':0.15,'eta0':0.001,
    'fit_intercept':True,'shuffle':True,'random_state':1,
}

gbm_param = {
    'n_estimators' :100, 'learning_rate':0.1, 'min_samples_split' :0.00001,
    'subsample':1.0, 'max_depth':5, 'max_features':0.4,
    'min_samples_leaf' :0.5, 'random_state' :1
}

lasso_params={
    'alpha':0.003,
    'normalize':True,
    'max_iter':200,'fit_intercept':True,'tol':0.007,
    'warm_start':True
}

ridge_params={
    'alpha':0.2,
    'normalize':True,
    'max_iter':200,'fit_intercept':False,'solver':'auto'
}

rf_params = {
    'criterion':'gini', 'max_leaf_nodes':30, 'n_estimators':1000, 'min_impurity_split':0.0000001,
    'max_features':0.25, 'max_depth':6, 'min_samples_leaf':20, 'min_samples_split':2,
    'min_weight_fraction_leaf':0.0, 'bootstrap':True,
    'random_state':1, 'verbose':False
    
}

### Model

In [ ]:
keras_model = stacking.KerasWrapper(ExtraTreesClassifier, None, params=rf_params)

In [ ]:
lgbm_model1 = stacking.LgbmWrapper(params=lgbm_params1, num_rounds = 1500, ealry_stopping=100,
                                   verbose_eval=False, base_score=True, maximize=False,
                                   y_value_log=False)

lgbm_model2 = stacking.LgbmWrapper(params=lgbm_params2, num_rounds = 1500, ealry_stopping=100,
                                   verbose_eval=False, base_score=True, maximize=False,
                                   y_value_log=False)

lgbm_model3 = stacking.LgbmWrapper(params=lgbm_params3, num_rounds = 1500, ealry_stopping=100,
                                   verbose_eval=False, base_score=True, maximize=False,
                                   y_value_log=False)

lgbm_model4 = stacking.LgbmWrapper(params=lgbm_params4, num_rounds = 1500, ealry_stopping=100,
                                   verbose_eval=False, base_score=True, maximize=False,
                                   y_value_log=False)

xgb_model1 = stacking.XgbWrapper(params=xgb_params1, num_rounds = 1500, ealry_stopping=100,
                                   verbose_eval=False, base_score=True, maximize=False,
                                   y_value_log=False)

xgb_model2 = stacking.XgbWrapper(params=xgb_params2, num_rounds = 1500, ealry_stopping=100,
                                   verbose_eval=False, base_score=True, maximize=False,
                                   y_value_log=False)

gbm_model = stacking.SklearnWrapper(clf = GradientBoostingClassifier,params=gbm_param)

ridge_model = stacking.SklearnWrapper(clf = Ridge,params=ridge_params)

lasso_model = stacking.SklearnWrapper(clf = Lasso,params=lasso_params)

rf_model = stacking.SklearnWrapper(clf = RandomForestClassifier,params=rf_params)

et_model = stacking.SklearnWrapper(clf = ExtraTreesClassifier,params=rf_params)

### Run!

In [ ]:
lgbm1_train,lgbm1_test = stacking.get_oof(lgbm_model1,x_train,y_train,x_test, log_loss, NFOLDS=5)
lgbm2_train,lgbm2_test = stacking.get_oof(lgbm_model2,x_train,y_train,x_test, log_loss, NFOLDS=5)
lgbm3_train,lgbm3_test = stacking.get_oof(lgbm_model3,x_train,y_train,x_test, log_loss, NFOLDS=5)
lgbm4_train,lgbm4_test = stacking.get_oof(lgbm_model4,x_train,y_train,x_test, log_loss, NFOLDS=5)
gc.collect()
xgb1_train,xgb1_test = stacking.get_oof(xgb_model1,x_train,y_train,x_test, log_loss, NFOLDS=5)
xgb2_train,xgb2_test = stacking.get_oof(xgb_model2,x_train,y_train,x_test, log_loss, NFOLDS=5)
gc.collect()
gbm1_train,gbm1_test = stacking.get_oof(gbm_model,x_train.fillna(-1),y_train,x_test.fillna(-1), log_loss,NFOLDS=5)
ridge_train,ridge_test = stacking.get_oof(ridge_model,x_train.fillna(-1),y_train,x_test.fillna(-1), log_loss,NFOLDS=5)
lasso_train,lasso_test = stacking.get_oof(lasso_model,x_train.fillna(-1),y_train,x_test.fillna(-1), log_loss,NFOLDS=5)
rf_train,rf_test = stacking.get_oof(rf_model,x_train.fillna(-1),y_train,x_test.fillna(-1), log_loss,NFOLDS=5)
et_train,et_test = stacking.get_oof(et_model,x_train.fillna(-1),y_train,x_test.fillna(-1), log_loss,NFOLDS=5)
gc.collect()

### Second Layer

In [ ]:
x_train_second_layer = np.concatenate((lgbm1_train, lgbm2_train, lgbm3_train,
                         lgbm4_train, xgb1_train, xgb2_train,
                         gbm1_train,ridge_train, lasso_train, rf_train,et_train), axis=1)
x_test_second_layer = np.concatenate((lgbm1_test, lgbm2_test, lgbm3_test,
                        lgbm4_test,xgb1_test,xgb2_test,
                       gbm1_test,ridge_test,lasso_test,rf_test,et_test), axis=1)

In [ ]:
lgbm_ex_no = 9
lgbm_meta_params = {
    'boosting':'gbdt', 'num_leaves':28, 'learning_rate':0.03, 'min_sum_hessian_in_leaf':0.1,
    'max_depth':7, 'feature_fraction':0.6, 'min_data_in_leaf':70, 'poission_max_delta_step':0.7,
    'bagging_fraction':0.8, 'min_gain_to_split':0, 'scale_pos_weight':1.0,
    'lambda_l2':0.1, 'lambda_l1':0.1, 'fair_c':1.0, 'bagging_freq':1,
    'objective':'binary', 'seed':1, 'categorical_feature':0, 'xgboost_dart_mode':False,
    'drop_rate':0.1, 'skip_drop':0.5, 'max_drop':50, 'top_rate':0.1, 'other_rate':0.1,
    'max_bin':255, 'min_data_in_bin':50, 'bin_construct_sample_cnt':1000000,
    'two_round':False, 'uniform_drop':False,'metric': 'binary_logloss','threads':6
}

lgbm_meta_model = stacking.LgbmWrapper(params=lgbm_meta_params, num_rounds = 2000, ealry_stopping=100,
               verbose_eval=False, base_score=True, maximize=False, y_value_log=False)

lgbm_cv_score,best_round = stacking.kfold_test(lgbm_meta_model, pd.DataFrame(x_train_second_layer), 
                                               y_train,  log_loss, NFOLDS=5 )


d_train_all = lgbm.Dataset(pd.DataFrame(x_train_second_layer), label=y_train)
bst = lgbm.train(lgbm_params4,d_train_all,best_round)
predictions = bst.predict(pd.DataFrame(x_test_second_layer))

In [ ]:
print("fscore result")
fscore_df = pd.concat([pd.DataFrame(bst.feature_name()),pd.DataFrame(bst.feature_importance())],axis=1)
fscore_df.columns = ['column','fscore']
fscore_df['fscore'] = fscore_df['fscore'].astype(int)
fscore_df.sort_values(by='fscore',ascending=False,inplace=True)
fscoe_output = 'fscore\\ex_'+str(lgbm_ex_no)+'_lgbm_fscore_'+ str(lgbm_cv_score)+ '.csv'
fscore_df.to_csv(fscoe_output)

fig, ax = plt.subplots(figsize=(12,18))
lgbm.plot_importance(bst, height=0.8, ax=ax)
plt.show()

In [ ]:
binary_predictions = [(0 if x < 0.5 else 1) for x in predictions]

In [ ]:
test = data_dummy.loc[data_dummy['PassengerId'] >= test_id]
submission = pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Survived": binary_predictions
})
submission.to_csv('./submission.csv', index=False)

In [ ]:
# print("Predict")
# sub = pd.read_csv('input/sample_submission.csv')
# for c in sub.columns[sub.columns != 'ParcelId']:
#     print(c)
#     sub[c] = predictions

# print("Wrting Files")
# sub_output = 'output\\ex_'+str(lgbm_ex_no)+'_lightgbm_'+str(lgbm_cv_score)
# sub.to_csv(sub_output+'.csv', index=False, float_format='%.4f') # Thanks to @inversion

# print("File Zip")
# jungle_zip = zipfile.ZipFile(sub_output +'.zip', 'w')
# jungle_zip.write(sub_output + '.csv', compress_type=zipfile.ZIP_DEFLATED)
 
# jungle_zip.close()
# print("Done")